In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [35]:
import copy

In [36]:
import io
import multiprocessing
import hyperopt.pyll

ModuleNotFoundError: No module named 'hyperopt'

In [7]:
from sklearn.model_selection import KFold
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict

from hyperopt import fmin, tpe, hp
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

cpu_count = 4
use_gpu = False

In [6]:
!pip install lightgbm xgboost catboost category-encoders 

     |████████████████████████████████| 65.7 MB 24 kB/s  eta 0:00:01
     |████████████████████████████████| 80 kB 448 kB/s eta 0:00:01
     |████████████████████████████████| 231 kB 474 kB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 289 kB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 52 kB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 394 kB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 627 kB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 387 kB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=dfb0a4c8dad0f87a7d647b6b177f0a0cbc6cd5cf958a1b93ae20730a8b127189
  Stored in directory: /home/asiwen/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [8]:
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

In [12]:
df_train.columns.to_list()

['continuous_annual_inc',
 'continuous_annual_inc_joint',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_dti_joint',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status',
 'continuous_mths_since_last_delinq',
 'continuous_mths_since_last_major_derog',
 'continuous_mths_since_last_record',
 'continuous_open_acc',
 'continuous_pub_rec',
 'discrete_addr_state_1_one_hot',
 'discrete_addr_state_2_one_hot',
 'discrete_addr_state_3_one_hot',
 'discrete_addr_state_4_one_hot',
 'discrete_addr_state_5_one_hot',
 'discrete_addr_state_6_one_hot',
 'discrete_addr_state_7_one_hot',
 'discrete_addr_state_8_one_hot',
 'discrete_addr_state_9_one_hot',
 'discrete_addr_state_10_one_hot',
 'discrete_addr_state_11_one_hot',
 'discrete_addr_s

In [16]:
# 50000 samples
df_train['loan_status'].value_counts()

1    39788
0    10212
Name: loan_status, dtype: int64

In [17]:
cols = df_train.columns.to_list()
x_cols = [v for v in cols if v not in ['loan_status']]

In [30]:
# 创建训练集
train_x = df_train.loc[:, x_cols]
train_y = df_train.loc[:, ['loan_status']]
train_dataset = lgb.Dataset(train_x, train_y)

In [36]:
test_x = df_test.loc[:, x_cols]
test_y = df_test.loc[:, ['loan_status']]
test_dataset = lgb.Dataset(test_x, test_y)

In [39]:

hyp_params = {'num_leaves': 31, 'objective': 'binary', 'metric':'binary_error'}
num_round = 2000

In [41]:
model = lgb.train(hyp_params, train_dataset, num_boost_round=2000, valid_sets=[train_dataset])

[LightGBM] [Info] Number of positive: 39788, number of negative: 10212
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024487 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2611
[LightGBM] [Info] Number of data points in the train set: 50000, number of used features: 141
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.795760 -> initscore=1.360002
[LightGBM] [Info] Start training from score 1.360002
[1]	training's binary_error: 0.20424
[2]	training's binary_error: 0.20424
[3]	training's binary_error: 0.20424
[4]	training's binary_error: 0.20424
[5]	training's binary_error: 0.14838
[6]	training's binary_error: 0.11022
[7]	training's binary_error: 0.099
[8]	training's binary_error: 0.09388
[9]	training's binary_error: 0.08814
[10]	training's binary_error: 0.08404
[11]	training's binary_error: 0.08162
[12]	training's binary_error: 0.0

In [33]:
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

NameError: name 'hp' is not defined

In [26]:
sts = kf.split(X)

array([1, 2])

In [28]:
params = {'a': 0, 'b': 1}

{'a': 0}